In [41]:
import torch
import torchvision
import torch.optim as optim
from data.data import get_train_test_loader
from model.network import FastUpdateNet, FastUpdateNetLarge
import torch.nn.functional as F

import pickle
import time 

In [26]:
n_epochs = 5
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
# torch.manual_seed(random_seed)

In [27]:
train_loader, test_loader = get_train_test_loader('fashionmnist')

In [28]:
train_losses = []
train_counter = []
test_losses = []
test_accuracies = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [50]:
from threading import Thread

def compute_back_M1(network):
  network.mNet1.backwardHidden()

def compute_back_M2(network):
  network.mNet2.backwardHidden()

def train(epoch, network):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    # print(network.mNet.saver.grad)
    # p1 = Thread(target=compute_back_M1, args=[network]) # start two independent threads
    # p2 = Thread(target=compute_back_M2, args=[network])
    
    # p1.start()
    # p2.start()
    network.mNet1.backwardHidden()
    network.mNet2.backwardHidden()
    # p1.join() # wait for the two threads to finish
    # return 
    # p2.join()

    correctness = (target == torch.argmax(output))
    optimizer.step()
    # network.mNet.weightUpdate(correctness, lr = learning_rate)
    if batch_idx % log_interval == 0:
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))


In [30]:
def test(network):
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  test_accuracies.append(100. * correct / len(test_loader.dataset))
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

## Test using 2 Ms (computing grad in main thread and separate thread)

In [51]:

import torch.nn as nn
fNet = FastUpdateNetLarge()
# fNet.mNet1 = torch.nn.Sequential(nn.Linear(392, 196), nn.ReLU(), nn.Linear(196, 98), nn.ReLU(), nn.Linear(98, 49), nn.ReLU())
# fNet.mNet2 = torch.nn.Sequential(nn.Linear(392, 196), nn.ReLU(), nn.Linear(196, 98), nn.ReLU(), nn.Linear(98, 49), nn.ReLU())
optimizer = optim.SGD(fNet.get_parameters(), lr=learning_rate, momentum=momentum)

In [52]:
train_losses = []
train_counter = []
test_losses = []
test_accuracies = []

s = time.perf_counter()

test(fNet)
for epoch in range(1, n_epochs + 1):
  train(epoch, fNet)
  test(fNet)
  # if (epoch + 1) % 10 == 0:
  #   torch.save(fNet, 'fNet-stdp-30000-epoch.pt')
  #   with open('fNet-stdp-3000-epoch-loss-accuracy.pkl', 'wb') as f:
  #     pickle.dump({'train_losses':train_losses, 'test_losses': test_losses, 'test_accuracies':test_accuracies}, f, protocol=pickle.HIGHEST_PROTOCOL)

# torch.save(fNet, 'fNet-stdp-30000-epoch.pt')
e = time.perf_counter()
print(f"Training time for {epoch} epochs: {e-s} seconds")

/home/kyle/projects/cs674/magic-m/model/network.py:173: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(o_5)
/home/kyle/pythonenvs/venv38/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3042, Accuracy: 1000/10000 (10%)


Test set: Avg. loss: 2.3020, Accuracy: 1011/10000 (10%)


Test set: Avg. loss: 2.2998, Accuracy: 1272/10000 (13%)


Test set: Avg. loss: 1.7716, Accuracy: 2012/10000 (20%)


Test set: Avg. loss: 0.8939, Accuracy: 6478/10000 (65%)


Test set: Avg. loss: 0.9403, Accuracy: 6217/10000 (62%)

Training time for 5 epochs: 90.69683430693112 seconds


In [34]:
test_accuracies

[tensor(10.),
 tensor(10.3800),
 tensor(18.2400),
 tensor(50.6500),
 tensor(59.6000),
 tensor(73.0700)]

## Test using no Ms, and only on main thread

In [45]:
def train_no_M(epoch, network):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    # print(network.mNet.saver.grad)
    # p1 = Thread(target=compute_back_M1, args=[network]) # start two independent threads
    # # p2 = Thread(target=compute_back_M2, args=[network])
    
    # p1.start()
    # # p2.start()
    # network.mNet2.backwardHidden()
    # p1.join() # wait for the two threads to finish
    # # return 
    # # p2.join()

    correctness = (target == torch.argmax(output))
    optimizer.step()
    # network.mNet.weightUpdate(correctness, lr = learning_rate)
    if batch_idx % log_interval == 0:
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [46]:

import torch.nn as nn
fNet = FastUpdateNetLarge()
fNet.mNet1 = torch.nn.Sequential(nn.Linear(392, 196), nn.ReLU(), nn.Linear(196, 98), nn.ReLU(), nn.Linear(98, 49), nn.ReLU())
fNet.mNet2 = torch.nn.Sequential(nn.Linear(392, 196), nn.ReLU(), nn.Linear(196, 98), nn.ReLU(), nn.Linear(98, 49), nn.ReLU())
optimizer = optim.SGD(fNet.parameters(), lr=learning_rate, momentum=momentum)

In [47]:
train_losses = []
train_counter = []
test_losses = []
test_accuracies = []

s = time.perf_counter()

test(fNet)
for epoch in range(1, n_epochs + 1):
  train_no_M(epoch, fNet)
  test(fNet)
  # if (epoch + 1) % 10 == 0:
  #   torch.save(fNet, 'fNet-stdp-30000-epoch.pt')
  #   with open('fNet-stdp-3000-epoch-loss-accuracy.pkl', 'wb') as f:
  #     pickle.dump({'train_losses':train_losses, 'test_losses': test_losses, 'test_accuracies':test_accuracies}, f, protocol=pickle.HIGHEST_PROTOCOL)

# torch.save(fNet, 'fNet-stdp-30000-epoch.pt')
e = time.perf_counter()
print(f"Training time for {epoch} epochs: {e-s} seconds")


Test set: Avg. loss: 2.3051, Accuracy: 1000/10000 (10%)


Test set: Avg. loss: 2.3021, Accuracy: 1000/10000 (10%)


Test set: Avg. loss: 2.2995, Accuracy: 1000/10000 (10%)


Test set: Avg. loss: 1.6677, Accuracy: 2402/10000 (24%)


Test set: Avg. loss: 0.8659, Accuracy: 6525/10000 (65%)


Test set: Avg. loss: 1.0468, Accuracy: 6144/10000 (61%)

Training time for 5 epochs: 49.55497955996543 seconds
